In [1]:
import numpy as np
import pandas as pd

In [ ]:
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201))

# Define values for k
k_values = [1, 2, 5, 10, 20, 50, 75, 100]

for i in range(201):
    data = phi[:,:,i]

    # Compute 1D (in r) and 2D FFT's
    fft_r = np.fft.fft(data)
    fft2 = np.fft.fft2(data)

    for k in k_values:
        # 1D
        # Keep only the top k frequencies
        fft_compressed1D = np.zeros_like(fft_r)
        fft_compressed1D[:k] = fft_r[:k]
        
        # Reconstruct the data using inverse FFT
        reconstructed1D = np.fft.ifft(fft_compressed1D).real
        
        # Compute the error (RMSE) and compression ratio
        diff1D = #TODO: Figure out what to subtract reconstructed1D from <---------------------------------------
        rmse1D = np.sqrt(np.mean(diff1D**2))
        cr1D = 101 / k

        # Add to dataframe
        cr_string1D = "CR1D_k=" + str(k)
        rmse_string1D = "RMSE1D_k=" + str(k)
        df.at[i, cr_string1D] = cr1D
        df.at[i, rmse_string1D] = rmse1D

        # 2D
        # Keep only the top k frequencies
        fft_compressed2D = np.zeros_like(fft2)
        fft_compressed2D[:k, :k] = fft2[:k, :k]
        
        # Reconstruct the data using inverse FFT
        reconstructed2D = np.fft.ifft2(fft_compressed2D).real
        
        # Compute the error (RMSE) and compression ratio
        diff2D = data - reconstructed2D
        rmse2D = np.sqrt(np.mean(diff2D**2))
        cr2D = (101 * 227) / (k * k)

        # Add to dataframe
        cr_string2D = "CR2D_k=" + str(k)
        rmse_string2D = "RMSE2D_k=" + str(k)
        df.at[i, cr_string2D] = cr2D
        df.at[i, rmse_string2D] = rmse2D

df